In [1]:
import requests
import json

import pandas as pd
from datetime import date, timedelta
from openhexa.sdk import current_run, workspace
from datetime import datetime, date, timedelta


This code is to process the ewars locations that were not matched to the dhis2 locations

In [2]:
# import the relevant parquets
full_pyramid = pd.read_parquet(
    "/home/leyregarrido/01_github_repos/openhexa-pipelines-drc-pnlp/dhis2_ewars_push/workspace/pipelines/dhis2_ewars_push/raw/pyramids/full_pyramid.parquet"
)
dhis2_pyramid = pd.read_parquet(
    "/home/leyregarrido/01_github_repos/openhexa-pipelines-drc-pnlp/dhis2_ewars_push/workspace/pipelines/dhis2_ewars_push/raw/pyramids/dhis2_pyramid.parquet"
)
raw_ewars_pyramid = pd.read_parquet(
    "/home/leyregarrido/01_github_repos/openhexa-pipelines-drc-pnlp/raw_ewars_pyramid.parquet"
)
ewars_pyramid = pd.read_parquet(
    "/home/leyregarrido/01_github_repos/openhexa-pipelines-drc-pnlp/dhis2_ewars_push/workspace/pipelines/dhis2_ewars_push/raw/pyramids/ewars_pyramid.parquet"
)

In [3]:
path_files = "/home/leyregarrido/01_github_repos/openhexa-pipelines-drc-pnlp/dhis2_ewars_push/workspace/not_found/in/"
processed = [
    "ewars_extract_20250527_2024Q1.parquet",
    "ewars_extract_20250527_2024Q2.parquet",
    "ewars_extract_20250527_2024Q3.parquet",
    "ewars_extract_20250527_2024Q4.parquet",
    "ewars_extract_20250527_2025Q1.parquet",
]
dhis2_repeated = [
    "dhis2_repeated_20250527_2024Q1.parquet",
    "dhis2_repeated_20250527_2024Q2.parquet",
    "dhis2_repeated_20250527_2024Q3.parquet",
    "dhis2_repeated_20250527_2024Q4.parquet",
    "dhis2_repeated_20250527_2025Q1.parquet",
]
not_matched = [
    "ewars_data_not_matched_20250527_2024Q1.parquet",
    "ewars_data_not_matched_20250527_2024Q2.parquet",
    "ewars_data_not_matched_20250527_2024Q3.parquet",
    "ewars_data_not_matched_20250527_2024Q4.parquet",
    "ewars_data_not_matched_20250527_2025Q1.parquet",
]

In [4]:
list_processed = []
for file in processed:
    path = path_files + file
    df = pd.read_parquet(path)
    list_processed.append(df)
processed = pd.concat(list_processed, ignore_index=True)
processed = processed.drop_duplicates()
list_dhis2_repeated = []
for file in dhis2_repeated:
    path = path_files + file
    df = pd.read_parquet(path)
    list_dhis2_repeated.append(df)
repeated = pd.concat(list_dhis2_repeated, ignore_index=True)
repeated = repeated.drop_duplicates()
list_not_matched = []
for file in not_matched:
    path = path_files + file
    df = pd.read_parquet(path)
    list_not_matched.append(df)
not_matched = pd.concat(list_not_matched, ignore_index=True)
not_matched = not_matched.drop_duplicates()

In [5]:
# Select the relevant not matched locations (The ones that appear in the processed file)
location_ids_not_matched = list(processed[processed["ewars_level_4_name_cleaned"].isna()]["location_id"].unique())
display(f"I have found {len(location_ids_not_matched)} locations in ewars that were not matched. ")
relevant_not_matched = not_matched[not_matched["level_4_id"].isin(location_ids_not_matched)].drop_duplicates()
display(f"{len(relevant_not_matched)} of these locations are in the not_matched_file.")

# Select the relevant repeated locations (The ones that appear in the processed file)
locations_dhis2_present = list(processed["dhis2_level_4_id"].unique())
possibly_relevant_repeated = repeated[repeated["dhis2_level_4_id"].isin(locations_dhis2_present)]
display(f"I have found {len(possibly_relevant_repeated)} repeated locations in DHIS2.")
to_be_rematched = [
    "HGR ALIMBONGO",
    "MBANGUMBAMU",
    "CSR BINGI",
    "CSR CHAMBUCHA",
    "HGR ITEBERO",
    "CSR KALIMBA",
    "KAILENGE",
    "CSR KITSOMBIRO",
    "CSR LUNYASENGE",
    "MIKUMBI",    
    "CSR MALEMBE",
    "CSR MUSENGE",
    "KINKUZU",
    "HGR VIRUNGA",
    "HGR WANGATA",
    "KIRIKIRI",
    "TEMBO",
]
relevant_repeated = possibly_relevant_repeated[possibly_relevant_repeated["ewars_level_4_name_cleaned"].isin(to_be_rematched)]
not_relevant_repeated = possibly_relevant_repeated[~possibly_relevant_repeated["ewars_level_4_name_cleaned"].isin(to_be_rematched)]

# Select the DHIS2 locations that were not matched
list_matched = list(processed["dhis2_level_4_id"].unique())
dhis2_not_matched = dhis2_pyramid[~dhis2_pyramid["level_4_id"].isin(list_matched)]

'I have found 13 locations in ewars that were not matched. '

'9 of these locations are in the not_matched_file.'

'I have found 36 repeated locations in DHIS2.'

In [6]:
in_not_matched = list(relevant_not_matched["level_4_id"].unique())
appears_misteriously = [element for element in location_ids_not_matched if element not in in_not_matched]
display(f"{len(appears_misteriously)} locations are not matched, but do not appear in the not_matched file.")
ewars_relevant = ewars_pyramid[ewars_pyramid["level_4_id"].isin(appears_misteriously)]
display(appears_misteriously)

'4 locations are not matched, but do not appear in the not_matched file.'

['08d2e793-838c-477a-b08f-f424fb98afb3',
 '25eb3236-89e7-47e2-8996-f6a22c664881',
 '2b06ed02-e60e-42f9-a968-676df527ac0a',
 '058d7da7-d6b6-4ed6-a09f-7379f05dee53']

In [7]:
# We need to format the relevant_repeated dataframe to match the processed dataframe
relevant_repeated_formated = relevant_repeated[
    [
        "ewars_level_4_name_cleaned",
        "ewars_level_4_id",
        "ewars_level_3_name_cleaned",
        "ewars_level_3_id",
        "ewars_level_2_name_cleaned",
        "ewars_level_2_id",
        "ewars_level_1_name_cleaned",
        "ewars_level_1_id",
    ]
]
relevant_repeated_formated = relevant_repeated_formated.rename(
    columns={
        "ewars_level_4_name_cleaned": "level_4_name_cleaned",
        "ewars_level_4_id": "level_4_id",
        "ewars_level_3_name_cleaned": "level_3_name_cleaned",
        "ewars_level_3_id": "level_3_id",
        "ewars_level_2_name_cleaned": "level_2_name_cleaned",
        "ewars_level_2_id": "level_2_id",
        "ewars_level_1_name_cleaned": "level_1_name_cleaned",
        "ewars_level_1_id": "level_1_id",

    }
)

In [8]:
# Revise that there are no locations in not_relevant_repeated that are actually relevant. We do this by eye.
display(not_relevant_repeated)

,ewars_level_4_name_cleaned,dhis2_level_4_name_cleaned,score_level_4,ewars_level_4_id,dhis2_level_4_id,ewars_level_3_name_cleaned,dhis2_level_3_name_cleaned,score_level_3,ewars_level_3_id,dhis2_level_3_id,ewars_level_2_name_cleaned,dhis2_level_2_name_cleaned,score_level_2,ewars_level_2_id,dhis2_level_2_id,ewars_level_1_name_cleaned,dhis2_level_1_name_cleaned,score_level_1,ewars_level_1_id,dhis2_level_1_id
0,BAHUMBU 1,BAHUMBU 1,100.0,d68f4c61-54e2-4be5-8910-95bbe6319fe9,egXYGA8gcJy,N'SELE,NSELE,90.909091,b292425a-1489-4713-bb88-cedfefc1d524,m0D9iUAjJ1G,KINSHASA,KINSHASA,100.0,54348a51-688c-4208-9a03-60a7f4a0d2a2,TwSa8zUu09Q,REPUBLIQUE DEMOCRATIQUE DU CONGO,REPUBLIQUE DEMOCRATIQUE DU CONGO,100.0,ac618e6f-bc64-4796-839a-853a4752c210,ymGeqzoPhN3
1,VIRUNGA,VIRUNGA,100.0,80f760c9-c25c-47d6-bef0-a1c0296f5445,iXAwuWUNsCl,KARISIMBI,KARISIMBI,100.000000,13d83b11-141e-4f93-b28d-1fef2237e5b5,RSaqeeEezZl,NORD KIVU,NORD KIVU,100.0,5ff00e7d-9b57-4cdc-9551-aa20632bae49,pIAYIpy4hiH,REPUBLIQUE DEMOCRATIQUE DU CONGO,REPUBLIQUE DEMOCRATIQUE DU CONGO,100.0,ac618e6f-bc64-4796-839a-853a4752c210,ymGeqzoPhN3
2,ALIMBONGO,ALIMBONGO,100.0,5964663c-d206-4fb4-bd10-e3d93c298838,VEhPr9jYMda,ALIMBONGO,ALIMBONGO,100.000000,eec2a16e-c34e-4520-ae61-aff93f0ef073,GLky3WLKZTi,NORD KIVU,NORD KIVU,100.0,5ff00e7d-9b57-4cdc-9551-aa20632bae49,pIAYIpy4hiH,REPUBLIQUE DEMOCRATIQUE DU CONGO,REPUBLIQUE DEMOCRATIQUE DU CONGO,100.0,ac618e6f-bc64-4796-839a-853a4752c210,ymGeqzoPhN3
3,BINGI,BINGI,100.0,ca6c3182-4ff1-46d9-898e-5f400a79c82b,IR5cJC0ktvE,ALIMBONGO,ALIMBONGO,100.000000,eec2a16e-c34e-4520-ae61-aff93f0ef073,GLky3WLKZTi,NORD KIVU,NORD KIVU,100.0,5ff00e7d-9b57-4cdc-9551-aa20632bae49,pIAYIpy4hiH,REPUBLIQUE DEMOCRATIQUE DU CONGO,REPUBLIQUE DEMOCRATIQUE DU CONGO,100.0,ac618e6f-bc64-4796-839a-853a4752c210,ymGeqzoPhN3
4,KITSOMBIRO,KITSOMBIRO,100.0,ccece595-6853-48a3-88c7-e587aa0466b5,wtCZgDia2HC,ALIMBONGO,ALIMBONGO,100.000000,eec2a16e-c34e-4520-ae61-aff93f0ef073,GLky3WLKZTi,NORD KIVU,NORD KIVU,100.0,5ff00e7d-9b57-4cdc-9551-aa20632bae49,pIAYIpy4hiH,REPUBLIQUE DEMOCRATIQUE DU CONGO,REPUBLIQUE DEMOCRATIQUE DU CONGO,100.0,ac618e6f-bc64-4796-839a-853a4752c210,ymGeqzoPhN3
5,KALIMBA,KALIMBA,100.0,5bdfb432-9605-4bab-b55b-b6edb98403cd,nivTO95GXtm,ALIMBONGO,ALIMBONGO,100.000000,eec2a16e-c34e-4520-ae61-aff93f0ef073,GLky3WLKZTi,NORD KIVU,NORD KIVU,100.0,5ff00e7d-9b57-4cdc-9551-aa20632bae49,pIAYIpy4hiH,REPUBLIQUE DEMOCRATIQUE DU CONGO,REPUBLIQUE DEMOCRATIQUE DU CONGO,100.0,ac618e6f-bc64-4796-839a-853a4752c210,ymGeqzoPhN3
6,LUNYASENGE,LUNYASENGE,100.0,e0370c46-8b07-4ab7-a8b2-854a82ebd4e8,bcsMMi4AwpE,ALIMBONGO,ALIMBONGO,100.000000,eec2a16e-c34e-4520-ae61-aff93f0ef073,GLky3WLKZTi,NORD KIVU,NORD KIVU,100.0,5ff00e7d-9b57-4cdc-9551-aa20632bae49,pIAYIpy4hiH,REPUBLIQUE DEMOCRATIQUE DU CONGO,REPUBLIQUE DEMOCRATIQUE DU CONGO,100.0,ac618e6f-bc64-4796-839a-853a4752c210,ymGeqzoPhN3
7,KIVIRIRI,KIVIRIRI,100.0,1a06f45f-2074-49c2-a3d3-8f586d40dcbe,cJSpUj7bgbF,LUBERO,LUBERO,100.000000,486eec71-29bc-49c3-b3b9-9da01aa20e1d,oTtGHeQSotN,NORD KIVU,NORD KIVU,100.0,5ff00e7d-9b57-4cdc-9551-aa20632bae49,pIAYIpy4hiH,REPUBLIQUE DEMOCRATIQUE DU CONGO,REPUBLIQUE DEMOCRATIQUE DU CONGO,100.0,ac618e6f-bc64-4796-839a-853a4752c210,ymGeqzoPhN3
8,ITEBERO,ITEBERO,100.0,59b61dfa-6bb4-4cdf-b780-e2740c0dab5b,mQK4EseMYth,ITEBERO,ITEBERO,100.000000,a2d140cc-6780-405b-90fe-85182e1b484d,bzo7dCPSXS2,NORD KIVU,NORD KIVU,100.0,5ff00e7d-9b57-4cdc-9551-aa20632bae49,pIAYIpy4hiH,REPUBLIQUE DEMOCRATIQUE DU CONGO,REPUBLIQUE DEMOCRATIQUE DU CONGO,100.0,ac618e6f-bc64-4796-839a-853a4752c210,ymGeqzoPhN3
9,MALEMBE,MALEMBE,100.0,0d6d73b6-ceb8-4f53-b7f7-42929fb6f393,sUg3bIvgG1V,ITEBERO,ITEBERO,100.000000,a2d140cc-6780-405b-90fe-85182e1b484d,bzo7dCPSXS2,NORD KIVU,NORD KIVU,100.0,5ff00e7d-9b57-4cdc-9551-aa20632bae49,pIAYIpy4hiH,REPUBLIQUE DEMOCRATIQUE DU CONGO,REPUBLIQUE DEMOCRATIQUE DU CONGO,100.0,ac618e6f-bc64-4796-839a-853a4752c210,ymGeqzoPhN3


In [9]:
# we concatenate the relevant not matched and the repeated relevant.
not_matched_total = pd.concat([relevant_not_matched, relevant_repeated_formated], ignore_index=True)
not_matched_total = not_matched_total[["level_4_name_cleaned", "level_4_id", "level_3_name_cleaned", "level_3_id", "level_2_name_cleaned", "level_2_id", "level_1_name_cleaned", "level_1_id"]]
display(not_matched_total)


,level_4_name_cleaned,level_4_id,level_3_name_cleaned,level_3_id,level_2_name_cleaned,level_2_id,level_1_name_cleaned,level_1_id
0,NSANDA,4e748420-204a-47e0-83c2-d840b9f5ecd7,GOMBE MATADI,057d806f-f25b-46ce-a527-27f6088a251d,KONGO CENTRAL,bb088b83-a7f6-4f04-b990-7f0772103eb1,REPUBLIQUE DEMOCRATIQUE DU CONGO,ac618e6f-bc64-4796-839a-853a4752c210
1,NTIMANSI,63a503e6-3080-4bb3-b43d-5c2bf06bbb4c,GOMBE MATADI,057d806f-f25b-46ce-a527-27f6088a251d,KONGO CENTRAL,bb088b83-a7f6-4f04-b990-7f0772103eb1,REPUBLIQUE DEMOCRATIQUE DU CONGO,ac618e6f-bc64-4796-839a-853a4752c210
2,SOMBALA,c4c7d1ac-de49-49d4-8672-a6aba7d97c3d,GOMBE MATADI,057d806f-f25b-46ce-a527-27f6088a251d,KONGO CENTRAL,bb088b83-a7f6-4f04-b990-7f0772103eb1,REPUBLIQUE DEMOCRATIQUE DU CONGO,ac618e6f-bc64-4796-839a-853a4752c210
3,YANDA,626a1dbf-139b-4b5a-bcee-c8316f734f6b,GOMBE MATADI,057d806f-f25b-46ce-a527-27f6088a251d,KONGO CENTRAL,bb088b83-a7f6-4f04-b990-7f0772103eb1,REPUBLIQUE DEMOCRATIQUE DU CONGO,ac618e6f-bc64-4796-839a-853a4752c210
4,KIHUMBA,749aa29e-c191-49f6-ade1-5534f9cae743,IDJWI,3fef9444-cadc-4159-9d81-c2fd8cb7bccb,SUD KIVU,1bcb9cba-2205-45fb-9812-3e1b5b487327,REPUBLIQUE DEMOCRATIQUE DU CONGO,ac618e6f-bc64-4796-839a-853a4752c210
5,CH MUHANGA,bb0e2dc6-06e3-4074-a95b-7b647e82ad86,ALIMBONGO,eec2a16e-c34e-4520-ae61-aff93f0ef073,NORD KIVU,5ff00e7d-9b57-4cdc-9551-aa20632bae49,REPUBLIQUE DEMOCRATIQUE DU CONGO,ac618e6f-bc64-4796-839a-853a4752c210
6,HGR OICHA,cb7949b3-e4ae-4d38-b3f0-8fd1d700e6e1,OICHA,a8331491-cd6f-4859-98e0-f0893a46b302,NORD KIVU,5ff00e7d-9b57-4cdc-9551-aa20632bae49,REPUBLIQUE DEMOCRATIQUE DU CONGO,ac618e6f-bc64-4796-839a-853a4752c210
7,FRATERNITE,e5ae31b5-c2b2-438a-8920-f6b17c21a229,MAKOTA,8e5b65c6-de71-4868-a070-0bc326682e9a,LOMAMI,c42cca0a-9da4-42d7-8a90-67e45ea5556a,REPUBLIQUE DEMOCRATIQUE DU CONGO,ac618e6f-bc64-4796-839a-853a4752c210
8,MAYUNGU,bd2c86c4-3a4e-47a6-9fad-2511f26b682a,KIMPUTU,bf379378-aa9d-4d56-a35c-7c61160fe053,KWILU,6cba73b3-77f0-4d5a-bc38-259e0753ae6f,REPUBLIQUE DEMOCRATIQUE DU CONGO,ac618e6f-bc64-4796-839a-853a4752c210
9,HGR ALIMBONGO,cc06a606-e086-4c81-abd2-77af3ad79663,ALIMBONGO,eec2a16e-c34e-4520-ae61-aff93f0ef073,NORD KIVU,5ff00e7d-9b57-4cdc-9551-aa20632bae49,REPUBLIQUE DEMOCRATIQUE DU CONGO,ac618e6f-bc64-4796-839a-853a4752c210


In [10]:
# We save the not matched total to a csv file. We can share this to ask for the matching
not_matched_total.to_csv("not_matched_ewars_locations.csv", index=False)